In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_table('../Datasets/Diabetic Retinopathy/messidor_features.arff', sep=',', index_col=False, header=None)

In [3]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# AUTOML

In [11]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [12]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7569444444444444

In [13]:
import pickle
with open('./models/diabetic_retinopathy_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

# RANDOM FOREST

In [30]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [31]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7083333333333334

# SVC

In [5]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             verbose=3)

In [7]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.78125